## Contents of this notebook

- 0: Load packages, scripts etc.
- 1: Construct trees. Uses the technology data stored in an .xlsx-file to construct dictionaries and other objects that fully characterize the production tree. These are then combined to an actual tree object using the nestingtree.nestingtree class.
- 2: Adding parameters to the database. Only some of the share-parameters etc. can be deduced directly from technology data. The rest of these as well as starting values for endogenous and exogenous variables must be collected in a database.
- 3: The model. Uses the tree object as well as the prepared database to construct a model using the *gmspython* class and its childclass *abate*.
The script ends by exporting the model as a pickle to be loaded in the calibration script.

### Loads

In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(py['main'])
import abatement, techdata_to_tree, sys, ShockFunction
os.chdir(curr)
data_folder = os.getcwd()+'\\Data'
gams_folder = data_folder + "\\..\\gamsmodels\\Main"
#Functions
def flatten_list(list_):
    return [item for sublist in list_ for item in sublist]

# **1: Construct trees**

#### Load technology data.
This runs the script in located in the file *techdata_to_tree.py*:

In [2]:
inputfile = "techdata_new2.xlsx"
output = techdata_to_tree.load_techcats(pd.read_excel(data_folder + "/" + inputfile, sheet_name=["inputdisp", "endofpipe", "inputprices"]))

#### The output of the code is a dictionary with three keys, referring to the two modules + a list of inputprices stored in the xlsx-file as well:

In [3]:
output.keys()

dict_keys(['ID', 'inputprices', 'EOP'])

The two different modules correspond to the two types of technology catalogs that the model can handle.\
*Input-displacing* (ID) and *End of pipe* (EOP).

In [4]:
modules = ["ID", "EOP"]

ID contains a dictionary related to input-displacing technologies, EOP does so for end of pipe (we use this naming convention throughout).\
They contain the following keys:

In [5]:
for module in modules:
    print("Keys of " + module + ": ", output[module].keys(), "\n")

Keys of ID:  dict_keys(['techs_inputs', 'techs', 'components', 'upper_categories', 'mu', 'Q2P', 'unit_costs', 'current_coverages', 'coverage_potentials', 'IO_tech', 'IO_tech_inputs', 'baseline_U_inputs']) 

Keys of EOP:  dict_keys(['techs_inputs', 'techs', 'components', 'upper_categories', 'mu', 'Q2P', 'unit_costs', 'current_coverages', 'coverage_potentials']) 



As an example, "techs" shows, for each technology, which technology good it produces, e.g. for input-displacing:

In [6]:
output["ID"]["techs"]

{'ID_1': ['U_ID_1_1', 'U_ID_1_2', 'U_ID_1_3'],
 'ID_2': ['U_ID_2_1', 'U_ID_2_2', 'U_ID_2_3'],
 'ID_3': ['U_ID_3_1', 'U_ID_3_2'],
 'ID_4': ['U_ID_4_1', 'U_ID_4_2', 'U_ID_4_3'],
 'ID_5': ['U_ID_5_1', 'U_ID_5_2']}

And "upper_categories" shows the mapping between energy services (E) and their respective components (C) for ID.\
All of the trees in the `output` object are dictionaries where the keys are nodes in the tree and the values are lists of connected branches.

In [7]:
output["ID"]["upper_categories"]

{'EL': ['C_EL_1', 'C_EL_2', 'C_EL_3', 'C_EL_4', 'C_EL_5', 'C_EL_base'],
 'EH': ['C_EH_1', 'C_EH_2', 'C_EH_3', 'C_EH_base'],
 'ER': ['C_ER_1', 'C_ER_2', 'C_ER_base']}

... and the mapping between emission types M ($CO_2$, $SO_2$ etc.) and the components C.\
Note importantly that this mapping does not constitute an actual part of the tree, because components are the outputs of the EOP sector.

In [8]:
output["EOP"]["upper_categories"]

{'CO2': ['C_CO2_1'],
 'SO2': ['C_SO2_1', 'C_SO2_2', 'C_SO2_3'],
 'NOX': ['C_NOX_1', 'C_NOX_2', 'C_NOX_3', 'C_NOX_4']}

### Initialize nesting tree, call it "Abatement"
The construction of the production tree starts with an initialization of the *nesting_tree* class:

In [9]:
nts = {"ID":nesting_tree.nesting_tree(name="ID"), "EOP":nesting_tree.nesting_tree(name="EOP")}

The "trees" attribute starts off empty, reflecting that no (sub)trees have been added yet. 

In [10]:
nts["ID"].trees

{}

#### The following cells add each of the subtrees to the tree-object, "nt"

##### First, input-displacing subtrees

Upper part, connecting energy services to components (this is an input-tree, so we do not supply additional keyword arguments).\
Again, we use the naming convention that a prefix reflects whether the tree is related to input-displacement (ID) or end of pipe (EOP).
The prefix is followed by an underscore and then letters reflecting which elements in the tree are connected.

The first tree connects energy services (E) to components (C), related to input-displacing technologies (ID)

In [23]:
nts["ID"].add_tree(output["ID"]["upper_categories"], tree_name = 'ID_EC', **{"type_f":"CES_norm"})

Before proceeding to the next tree, lets print some information from this tree.\
First, we see that the tree has been added to the list of trees in the "aggregate" tree, nt:

In [24]:
nts["ID"].trees

{'ID_EC': <nesting_trees.nt at 0x228f796ae08>}

Second, information about the tree is automatically added when the tree is added, e.g. the "type_f" attribute states that the functional form in this subtree is constant elasticity of substitution (CES)

In [32]:
nts["ID"].trees["ID_EC"].__dict__

{'name': 'ID_EC',
 'tree': {'EL': ['C_EL_1',
   'C_EL_2',
   'C_EL_3',
   'C_EL_4',
   'C_EL_5',
   'C_EL_base'],
  'EH': ['C_EH_1', 'C_EH_2', 'C_EH_3', 'C_EH_base'],
  'ER': ['C_ER_1', 'C_ER_2', 'C_ER_base']},
 'type_io': 'input',
 'version': 'std',
 'n': 'n',
 'nn': 'nn',
 'nnn': 'nnn',
 'nnnn': 'nnnn',
 'nnnnn': 'nnnnn',
 'map_': 'map_ID_EC',
 'kno': 'kno_ID_EC',
 'bra': 'bra_ID_EC',
 'inp': 'inp_ID_EC',
 'out': 'out_ID_EC',
 'temp_namespace': None,
 'type_f': 'CES_norm',
 'database': <DataBase.GPM_database at 0x228f796a648>}

However, even though the namespace includes e.g. "map_":"map_ID_EC", the database of the tree still does not contain this mapping. This requires running the run() method.\
Instead of doing that for each tree individually, we instead add all trees and call the run_all() methods which also calls the run() on each tree.

Next, we add the middle part, connecting components C to their technology goods U

In [33]:
nts["ID"].add_tree(output["ID"]["components"], tree_name = "ID_CU", **{"type_f":"MNL"})

The two remaining parts are the bottom ones. \
First, the one that connects technologies to their outputs (technology goods). That these are outputs is specified explicitly by using the `type_io` keyword.\
Second, the one that connects technologies to their inputs (non-capital inputs X, and capital K)

In [34]:
nts["ID"].add_tree(output["ID"]["techs"], tree_name="ID_TU", **{'type_io': 'output', 'type_f': 'CET_norm'})
nts["ID"].add_tree(output["ID"]["techs_inputs"], tree_name="ID_TX")

Baseline components and technology goods, and their respective sets of inputs are also part of the aggregate tree. We distinguish between those with their own inputs (these baseline technologies are the ones that come from knowing which energy mix a (set of) technologies replaces. The remaining baseline technology goods (and all baseline components by construction) are outputs of the "IO technology". This IO technology in turn draws on all inputs from the economy. This is the one we calibrate to make sure we replicate IO data. 

In [35]:
nts["ID"].add_tree(output["ID"]["IO_tech"], tree_name="ID_IOCU", **{"type_io":"output", "type_f":"CET_norm"})
nts["ID"].add_tree(output["ID"]["IO_tech_inputs"], tree_name="ID_IOX")

In [36]:
if output["ID"]["baseline_U_inputs"]:
    nts["ID"].add_tree(output["ID"]["baseline_U_inputs"], tree_name="ID_UbaseX")
else:
    print("No replacing baseline technology goods")

#### Next, we add the end of pipe subtrees.
First, the one connecting components (which are the final product in the end-of-pipe tree) to the technology goods from which they are created:

In [37]:
nts["EOP"].add_tree(output["EOP"]["components"], tree_name = "EOP_CU", **{"type_f":"MNL"})

Second, the bottom part. This consists of technologies and their outputs and inputs respectively, similarly to with input-displacing technologies:

In [38]:
nts["EOP"].add_tree(output["EOP"]["techs"], tree_name="EOP_TU", **{'type_io': 'output', 'type_f': 'CET_norm'})
nts["EOP"].add_tree(output["EOP"]["techs_inputs"], tree_name="EOP_TX")

#### Trees related to final goods: One taking energy services and remaining inputs to produce a composite good Y, and one splitting Y into sX

In [39]:
Y_inputs = list(output["ID"]["upper_categories"].keys())
Y_outputs = ["Y_out_" + c for c in output["inputprices"].keys()[:-1]]

In [40]:
Q2P_addition = []
for inp in output["inputprices"].index[:-1]:
    Y_inputs += ["Y_in_" + inp]
    Q2P_addition.append(("Y_in_" + inp, inp))

In [41]:
output["ID"]["Q2P"] = output["ID"]["Q2P"].append(pd.MultiIndex.from_tuples(Q2P_addition, names=["n", "nn"]))

In [42]:
#Share parameters are simply 1/N on both the input and output side
for inp in Y_inputs:
    output["ID"]["mu"][(inp, "Y")] = 1/len(Y_inputs)
for out in Y_outputs:
    output["ID"]["mu"][(out, "Y")] = 1/len(Y_outputs)

In [43]:
nts["ID"].add_tree({"Y":Y_inputs}, tree_name="ID_Y_in")
nts["ID"].add_tree({"Y":Y_outputs}, tree_name="ID_Y_out", **{'type_io': 'output', 'type_f': 'CET'})

##### Now, all trees have been added:

In [44]:
print(nts["ID"].trees, "\n", nts["EOP"].trees)

{'ID_EC': <nesting_trees.nt object at 0x00000228F796AE08>, 'ID_CU': <nesting_trees.nt object at 0x00000228F7971E48>, 'ID_TU': <nesting_trees.nt object at 0x00000228F789CF48>, 'ID_TX': <nesting_trees.nt object at 0x00000228F789C2C8>, 'ID_IOCU': <nesting_trees.nt object at 0x00000228F791FE08>, 'ID_IOX': <nesting_trees.nt object at 0x00000228F791F408>, 'ID_UbaseX': <nesting_trees.nt object at 0x00000228F7886EC8>, 'ID_Y_in': <nesting_trees.nt object at 0x00000228F79628C8>, 'ID_Y_out': <nesting_trees.nt object at 0x00000228F7962548>} 
 {'EOP_CU': <nesting_trees.nt object at 0x00000228F7962C88>, 'EOP_TU': <nesting_trees.nt object at 0x00000228F796A408>, 'EOP_TX': <nesting_trees.nt object at 0x00000228F796AE48>}



The next step is to use the method `run_all`. This runs through a number of steps, where it sets up sets/subsets/mappings identifying which elements e.g. are inputs, intermediate goods, and outputs in the aggregate tree, i.e. the combination of the subtrees we just added. *Tutorial_nesting_tree* includes a brief review of these.\
There are still some of the objects in `output` that we have not used. We will return to these later as they become relevant.

In [45]:
nts["ID"].__dict__

{'name': 'ID',
 'version': 'std',
 'trees': {'ID_EC': <nesting_trees.nt at 0x228f796ae08>,
  'ID_CU': <nesting_trees.nt at 0x228f7971e48>,
  'ID_TU': <nesting_trees.nt at 0x228f789cf48>,
  'ID_TX': <nesting_trees.nt at 0x228f789c2c8>,
  'ID_IOCU': <nesting_trees.nt at 0x228f791fe08>,
  'ID_IOX': <nesting_trees.nt at 0x228f791f408>,
  'ID_UbaseX': <nesting_trees.nt at 0x228f7886ec8>,
  'ID_Y_in': <nesting_trees.nt at 0x228f79628c8>,
  'ID_Y_out': <nesting_trees.nt at 0x228f7962548>},
 'database': <DataBase.GPM_database at 0x228f78c55c8>}

In [46]:
standard_sets = ('inp','out','int','wT','map_all','kno_out','kno_inp')
for module in modules:
    namespace = {k: module + '_' + k for k in standard_sets}
    nts[module].run_all(**namespace)
    #Also replaces keys with module-specific names, i.e. changes 'inp' to 'ID_inp' in the attributes/keys.
    for std_set in standard_sets:
        if hasattr(nts[module], std_set):
            setattr(nts[module], module + "_" + std_set, getattr(nts[module], std_set))
            delattr(nts[module], std_set)

This constructs the aggregate tree. \
For a few highlights, let's check:
1. The outputs of the aggregate tree (aggregate simply refers to the combination of all the individual (sub)trees\
This includes energy services (from ID) and components (from EOP)

In [47]:
list(nts["ID"].database.series["ID_out"].vals)

['Y_out_inp5', 'Y_out_oil', 'Y_out_inp4', 'Y_out_inp3', 'Y_out_electricity']

2. The outputs of a particular tree, say EOP_TU. Note that in this particular case, the reported list is empty. This is because an 'output' refers to whether it is an output of the aggregate tree and not just this particular one.

In [48]:
list(nts["EOP"].trees["EOP_TU"].database.series["out_EOP_TU"].vals)

['EOP_t5', 'EOP_t4', 'EOP_t2', 'EOP_t3', 'EOP_t1']

3. Likewise, we can check the inputs of EOP_TU\
This reports that the Us are inputs, which they really are not. This is not a problem, because the tree is constructed in the correct way. But not that U's are not inputs. Not of the aggregate tree (in which Us are intermediates), but not in the TU-tree either, since here, they are outputs (they are produced by technologies).

In [49]:
nts["EOP"].trees["EOP_TU"].database.series["inp_EOP_TU"].vals

Index(['U_EOP_t2_1', 'U_EOP_t4_1', 'U_EOP_t2_3', 'U_EOP_t1_1', 'U_EOP_t2_2',
       'U_EOP_t4_2', 'U_EOP_t3_1', 'U_EOP_t5_2', 'U_EOP_t5_1', 'U_EOP_t1_2'],
      dtype='object', name='n')

Lastly, lets print the objects that the `nt` class instance contains:

In [50]:
nts["ID"].__dict__

{'name': 'ID',
 'version': 'std',
 'trees': {'ID_EC': <nesting_trees.nt at 0x228f796ae08>,
  'ID_CU': <nesting_trees.nt at 0x228f7971e48>,
  'ID_TU': <nesting_trees.nt at 0x228f789cf48>,
  'ID_TX': <nesting_trees.nt at 0x228f789c2c8>,
  'ID_IOCU': <nesting_trees.nt at 0x228f791fe08>,
  'ID_IOX': <nesting_trees.nt at 0x228f791f408>,
  'ID_UbaseX': <nesting_trees.nt at 0x228f7886ec8>,
  'ID_Y_in': <nesting_trees.nt at 0x228f79628c8>,
  'ID_Y_out': <nesting_trees.nt at 0x228f7962548>},
 'database': <DataBase.GPM_database at 0x228f78c55c8>,
 'n': 'n',
 'nn': 'nn',
 'nnn': 'nnn',
 'nnnn': 'nnnn',
 'nnnnn': 'nnnnn',
 'fg': 'fg',
 'prune_trees': {'OnlyQ', 'bra', 'inp', 'kno', 'out'},
 'ID_inp': 'ID_inp',
 'ID_out': 'ID_out',
 'ID_int': 'ID_int',
 'ID_wT': 'ID_wT',
 'ID_map_all': 'ID_map_all',
 'ID_kno_out': 'ID_kno_out',
 'ID_kno_inp': 'ID_kno_inp'}

## 1.5: Construct some sets needed before calculating starting values

In [56]:
def multiindex_series(idx_level_names, idx_name=None, series_name=None):
    if idx_name is None and series_name is not None:
        idx_name = series_name
    elif idx_name is not None and series_name is None:
        series_name = idx_name
    elif idx_name is None and series_name is None:
        raise Exception("Supply either index name or series name")
    idx = pd.MultiIndex(levels=[[]]*len(idx_level_names), codes=[[]]*len(idx_level_names), names=idx_level_names)
    ser = pd.Series(index=idx, dtype=float)
    ser.rename(series_name, inplace=True)
    #ser.index.name = idx_name
    return ser

In [57]:
def find_key_from_value(d, value):
    assert isinstance(d, dict)
    out = []
    for (k, v) in d.items():
        if value in v:
            out.append(k)
    if len(out) == 1:
        return out[0]
    else:
        raise Exception("Value exists in multiple keys")

In [58]:
def find_true_input(inp, Q2P):
    true = list(Q2P[Q2P.get_level_values(0).isin([inp])].get_level_values(1))[0]
    return true

In [59]:
#sumX. The sum of energy use for baseline technologies (IO_tech + replacing baselines)
sumX2X = {}
sumX = {"ID":multiindex_series(idx_level_names=["n", "nn"], series_name="sumX"), "EOP":multiindex_series(idx_level_names=["n", "nn"], series_name="sumX")}
for x in output["ID"]["IO_tech_inputs"]["IO_tech"]:
    inp = find_true_input(x, output["ID"]["Q2P"])
    sumX["ID"][(inp, x)] = np.nan
#pd.IndexSlice[pd.Series(mu.index.get_level_values(1).isin(output["ID"]["techs"].keys())), pd.Series(mu.index.get_level_values(1).isin(output["ID"]["techs"].keys()))]
#output["ID"]["Q2P"]
for module in modules:
    for t in output[module]["techs_inputs"]:
        for x in output[module]["techs_inputs"][t]:
            sumX[module][(find_true_input(x, output[module]["Q2P"]), x)] = np.nan

for baseU in output["ID"]["baseline_U_inputs"]:
    for x in output["ID"]["baseline_U_inputs"][baseU]:
        sumX["ID"][(find_true_input(x, output["ID"]["Q2P"]), x)] = np.nan

for Y_inp in Y_inputs:
    if Y_inp not in output["ID"]["upper_categories"]:
        #if not an E, add to sumX2X
        sumX["ID"][(find_true_input(Y_inp, output["ID"]["Q2P"]), Y_inp)] = np.nan

#change name of the sum of an input to "sum_x"
sumX = {"ID":sumX["ID"].reset_index(), "EOP":sumX["EOP"].reset_index()}
for module in modules:
    sumX[module]["n"] = "sum_" + sumX[module]["n"]
    sumX2X[module] = sumX[module].set_index(["n", "nn"])["sumX"].index

In [60]:
phi = multiindex_series(idx_level_names=["n", "nn"], series_name="phi")
M0 = pd.Series([], name="M0", dtype="float64")
M = pd.Series([], name="M", dtype="float64")
#Here sets emission intensity equal to 0.1 by construction for now

In [61]:
#add each input to phi as well (needed for the Phat equation)
default_emsint = 0
emission_intensities = {
    "oil":{
        "CO2":0.2,
        "NOX":0.2
    },
    "electricity":{
        "SO2":0.2
    }
}
for m in output["EOP"]["upper_categories"].keys():
    M0[m] = 5 #Emissions default simply chosen to be 5
    M[m] = 5
    for module in modules:
        for x in sumX2X[module].get_level_values(1):
            true_input = find_true_input(x, output[module]["Q2P"])
            if (true_input in emission_intensities and m in emission_intensities[true_input]):
                phi[(m, x)] = emission_intensities[true_input][m]
            else:
                phi[(m, x)] = default_emsint

In [62]:
#Emission prices,  and theta which measures the potentials of components.
pM = M.copy()
pM[:] = 50
pM.name = "pM"
pM.index.name = "n"

pMhat = M.copy()
pMhat[:] = 50
pMhat.name = "pMhat"
pMhat.index.name = "n"

# **2: Adding parameters to the database**

The next step is to construct a database that contains all the share-parameters that we can deduce from technology data, as well as appropriate starting values for endogenous variables. These starting values will be set to the value that they would have if the entire tree was Leontief.
The end goal is to make sure the database contained by `nt` includes all these parameters and starting values. 
To do so, we construct an empty database, add the share-parameters and starting values to this, and then finally merge that database with the one already in `nt`.

The GPM database of the nesting_tree instance (`nt`) does not contain the $\mu$ parameters calculated from technology data yet.\
The $\mu$-parameters are stored under the key of the same name in `output`.\
Let's check out the database of `nt` now, to see that it does not include any symbol called $\mu$ yet:

In [63]:
print("The database nt.database is of type " + str(type(nts["ID"].database)))
print("Does the database include a symbol called mu? ") 
if "mu" in nts["ID"].database.series:
    print("YES!")
else:
    print("NO!")

The database nt.database is of type <class 'DataBase.GPM_database'>
Does the database include a symbol called mu? 
NO!


First we create an empty database:

In [64]:
dbs = {"ID":DataBase.GPM_database(), "EOP":DataBase.GPM_database()}

We store the share-parameters from technology data in an object called `mu`. This does not contain all share-parameters of the tree (e.g. it does not include the share parameters of the inputs under baseline technology goods, or the share parameters for technology goods under components).

In [65]:
mu = {"ID":output["ID"]["mu"].copy(), "EOP":output["EOP"]["mu"].copy()}
# for module in modules:
#     mu[module].name = module + "_" + mu[module].name

Check out the mu parameters (prints just the five first entries here).\
Note: The mu-object is a series with a multiindex, where the first level is called `n` and the second level `nn`. The first level contains branches and the second level contains nodes.

In [66]:
mu["ID"].head()

n                 nn  
ID_1_electricity  ID_1    0.475
ID_1_oil          ID_1    0.475
ID_1_K            ID_1    4.050
C_EL_1            EL      0.050
U_ID_1_1          ID_1    0.500
Name: mu, dtype: float64

#### Calculate starting values (corresponding to the solution if the entire tree was Leontief).
For demands/quantities, we use the fact that Leontief demand is given by 
$$q_j = \mu_j \left(\frac{p_i}{p_j}\right)^\sigma q_i = \mu_j q_i$$ for CES demand where $i$ refers to a node and $j$ refers to the branch. The CET and the MNL form is identical under Leontief.\
For prices, we generally use that when we know the price of each branch under a node as well as the quantities of these branches, the zero profit condition can be rearranged to give us the price of the node:
$$ q_i p_i = \sum_j q_j p_j \quad \Leftrightarrow \quad  p_i = \frac{\sum_j q_j p_j}{q_i}$$

Output quantities are held in the `qS` object. Output quantities are held fixed in the partial equilibrum (the output prices are the variables that adjust), and we simply set them to `output_quantity` in this example.

In [67]:
output_quantity_Yout = 100 * len(Y_inputs) / len(Y_outputs) #Makes qD[E] = 100 by construction
output_quantity_M = 100
qS = {"ID":pd.Series([], name="qS", dtype="float64"), "EOP":pd.Series([], name="qS", dtype="float64")}

`qS` is now an empty series to which we add the value of 100 for each output of the tree:

In [68]:
for nt in nts:
    for out in nts[nt].database.series[nt+"_out"]:
        if nt == "ID":
            quantity = output_quantity_Yout
        elif nt == "EOP":
            quantity = output_quantity_M
        qS[nt][out] = quantity

All other quantities are kept in the object `qD`, which we initialize:

In [69]:
qD = {"ID":pd.Series([], name="qD", dtype="float64"), "EOP":pd.Series([], name="qD", dtype="float64")}

First, we calculate the composite Y and the quantities of the inputs for Y (E + dX rest)

In [70]:
qD["ID"]["Y"] = sum(qS["ID"])

In [71]:
for Yinp in Y_inputs:
    qD["ID"][Yinp] = mu["ID"].loc[(Yinp, "Y")] * qD["ID"]["Y"]

Second, we calculate components' starting values for input-displacing (we did it for EOP with qS, because components are outputs there). \
We use the demand equation stated earlier (multiplication of share-parameter and relevant node/component). This calculates quantities for all components, including baseline components

In [72]:
for E in output["ID"]["upper_categories"]:
    for C in output["ID"]["upper_categories"][E]:
        qD["ID"][C] = mu["ID"].loc[(C, E)] * qD["ID"][E]

Next, we use that we have estimates of U to set these (current coverages split according to overlap), and then calculate the $\mu$s residually

In [73]:
for index, curr_coverage in output["ID"]["current_coverages"].iteritems():
    qD["ID"][index[0]] = curr_coverage * qD["ID"][index[1]]

The $\mu$s of technology goods (in their component nest) residually. This includes the share-parameters for the baseline technology goods $\bar U$. Since we still have not set the starting values for the quantities of these baseline technology goods, we set these using the share-parameters that are calculated here as well.

In [74]:
for C in output["ID"]["components"]:
    #The ':-1' leaves out the baseline technology good, which we handle separately afterwards
    nonbase_U = output["ID"]["components"][C][:-1]
    base_mu = 1
    for U in nonbase_U:
        mu["ID"][(U, C)] =  qD["ID"][U]/qD["ID"][C]
        base_mu -= mu["ID"][(U, C)]
    #Quantities of baseline U
    mu["ID"][(output["ID"]["components"][C][-1], C)] = base_mu
    qD["ID"][output["ID"]["components"][C][-1]] = base_mu * qD["ID"][C]
    if base_mu <= 0:
        print(C)
        print(nonbase_U)
        raise Exception("base_mu is not positive")


For end of pipe, we simply set the share-parameters of the technology goods under components as $1/N$ where $N$ is the number of branches/technology goods under a given component. Having set these, we can calculate the corresponding quantities using the CES Leontief demand equation.

In [75]:
for C in output["EOP"]["components"]:
    for U in output["EOP"]["components"][C]:
        mu["EOP"].loc[(U, C)] = 1/len(output["EOP"]["components"][C])
        qD["EOP"][U] = mu["EOP"].loc[(U, C)] * qS["EOP"][C]

Then, we calculate the starting values of technologies $\tau$ as the sum of their relevant $U$, since the CET function should be scale-preserving.\
The drawback here, for EOP only, is that the quantities of $U$ do not necessarily adhere to the relative sizes of their share-parameters (which are set based on the technology catalog). 

In [76]:
for module in modules:
    for tech in output[module]["techs"]:
        qD[module][tech] = qD[module][output[module]["techs"][tech]].sum()

Lastly, we set the starting values of the inputs $X$ that go into technologies. These again use the demand equation. The share-parameters of these were given directly from technology data.

In [77]:
for module in modules:
    techs = output[module]["techs_inputs"]
    for tech in techs:
        for inp in techs[tech]:
            qD[module][inp] = mu[module].loc[(inp, tech)] * qD[module][tech]

Set the quantity of the IO-technology as the sum of the quantities of the goods ($U_0$ and $C_0$) that it provides. Afterwards, calculate the relevant share parameters using the fraction of IO-tech to the goods. 

In [78]:
#Quantity of the IO technology
qD["ID"]["IO_tech"] = 0
for base in output["ID"]["IO_tech"]["IO_tech"]:
    qD["ID"]["IO_tech"] += qD["ID"][base]
#share parameters for each the non-replacing baseline technologies:
for base in output["ID"]["IO_tech"]["IO_tech"]:
    mu["ID"][(base, "IO_tech")] = qD["ID"][base] / qD["ID"]["IO_tech"]

##### Prices:

The next part contains the setting of starting values for prices.
The prices of inputs (the very bottom of the tree) are fully exogenous, whereas all other prices are endogenous. The prices of outputs (those whose quantities are contained in `qS` are kept in the object `PbT` (refers to "prices before taxes") whereas the rest of the prices are kept in `PwT`.

Objects for storing prices of goods:

In [79]:
PwThat = {"ID":pd.Series([], name="PwThat", dtype="float64"), "EOP":pd.Series([], name="PwThat", dtype="float64")}
PwT = {"ID":pd.Series([], name="PwT", dtype="float64"), "EOP":pd.Series([], name="PwT", dtype="float64")}
PbT = {"ID":pd.Series([], name="PbT", dtype="float64"), "EOP":pd.Series([], name="PbT", dtype="float64")}

Prices of inputs are also stated in the catalog:

In [80]:
for module in modules:
    for t, inputs in output[module]["techs_inputs"].items():
        for inp in inputs:
            PwT[module][inp] = output["inputprices"][inp.split("_")[-1]]
            p = output["inputprices"][inp.split("_")[-1]]
            #Add emission prices to Phat
            taxes = 0
            for m in pM.index:
                taxes += phi[(m, inp)] * pM[m]
            PwThat[module][inp] = p + taxes

In [81]:
#Prices on the dX in final goods production
for Y_inp in Y_inputs:
    if Y_inp not in output["ID"]["upper_categories"].keys():
        p = output["inputprices"][find_true_input(Y_inp, output["ID"]["Q2P"])]
        PwT["ID"][Y_inp] = p
        taxes = 0
        for m in pM.index:
            taxes += phi[(m, Y_inp)] * pM[m]
        PwThat["ID"][Y_inp] = p + taxes

Add the prices of technologies: $p^\tau$. These correspond to the weighted average of the prices of its inputs.

In [82]:
for module in modules:
    for t, inputs in output[module]["techs_inputs"].items():
        PwThat[module][t] = pd.concat([PwThat[module][inputs], qD[module][inputs]], axis=1).product(axis=1).sum() / qD[module][t]

These prices are equal to unit costs by construction, which we check here for good measure. The differences between prices and unit costs are:

In [83]:
round(output["ID"]["unit_costs"].set_index("tech")["unit_cost"] - PwThat["ID"][output["ID"]["unit_costs"]["tech"]], 2)

tech
ID_1   -14.25
ID_2   -12.10
ID_3     0.00
ID_4    -2.00
ID_5    -8.00
dtype: float64

For $p^U$, we simply assume $p^U = p^\tau$, which, since the share-parameters in the CET nest splitting $\tau$ into its produced technology goods sum to one, is consistent with zero profits (mroe generally, scale preservation ensures zero profits with this assumption)

In [84]:
for module in modules:
    for t in output[module]["techs"]:
        for U in output[module]["techs"][t]:
            PwThat[module][U] = PwThat[module][t]

Prices on components for end of pipe should be allocated to PbT because that includes the prices of outputs:

In [85]:
for C, U_list in output["EOP"]["components"].items():
    PbT["EOP"][C] = pd.concat([PwThat["EOP"][U_list], qD["EOP"][U_list]], axis=1).product(axis=1).sum() / qS["EOP"][C]

Insert baseline input quantities and baseline technology good prices (only relevant for ID since baselines do not exist in EOP).

This also (for now, should be set with input-output macro data later) sets the share-parameters of the inputs for the baseline technology goods. It assumes that the baseline uses all energy inputs in equal proportions and that these share-parameters sum to 1. Using these share-parameters, the corresponding quantities are added to `qD`.

In [86]:
inputs = output["ID"]["IO_tech_inputs"]["IO_tech"]
for inp in inputs:
    mu["ID"][(inp, "IO_tech")] = 1/len(inputs)
    PwT["ID"][inp] = output["inputprices"][inp.split("_")[-1]]
    p = output["inputprices"][inp.split("_")[-1]]
    taxes = 0
    for m in pM.index:
        taxes += phi[(m, inp)] * pM[m]
    PwThat["ID"][inp] = p + taxes
    qD["ID"][inp] = mu["ID"][(inp, "IO_tech")] * qD["ID"]["IO_tech"]
#Price of the IO technology
PwThat["ID"]["IO_tech"] = pd.concat([PwThat["ID"][inputs], qD["ID"][inputs]], axis=1).product(axis=1).sum() / qD["ID"]["IO_tech"]

In [87]:
for base in output["ID"]["IO_tech"]["IO_tech"]:
    PwThat["ID"][base] = PwThat["ID"]["IO_tech"] 

Quantities and prices of inputs to replacement-baseline-technologies and the cost of these as well:

In [88]:
for base_U, inputs in output["ID"]["baseline_U_inputs"].items():
    for inp in inputs:
        qD["ID"][inp] = mu["ID"][(inp, base_U)] * qD["ID"][base_U]
        PwT["ID"][inp] = output["inputprices"][inp.split("_")[-1]] #Actual prices
        p = output["inputprices"][inp.split("_")[-1]]
        taxes = 0
        for m in pM.index:
            taxes += phi[(m, inp)] * pM[m]
        PwThat["ID"][inp] = p + taxes
    PwThat["ID"][base_U] =  pd.concat([PwThat["ID"][inputs], qD["ID"][inputs]], axis=1).product(axis=1).sum() / qD["ID"][base_U]

Prices of components:

In [89]:
for C, U_list in output["ID"]["components"].items():
    PwThat["ID"][C] = pd.concat([PwThat["ID"][U_list], qD["ID"][U_list]], axis=1).product(axis=1).sum() / qD["ID"][C]

Prices of energy services ("upper categories" in ID). 

In [90]:
for E, C_list in output["ID"]["upper_categories"].items():
    PwThat["ID"][E] = pd.concat([PwThat["ID"][C_list], qD["ID"][C_list]], axis=1).product(axis=1).sum() / qD["ID"][E]

Price of composite of Y and outputs of Y

In [91]:
PwThat["ID"]["Y"] = pd.concat([PwThat["ID"][Y_inputs], qD["ID"][Y_inputs]], axis=1).product(axis=1).sum() / qD["ID"]["Y"]

In [92]:
#Price of final goods are equal to price of Y
for Y_out in Y_outputs:
    PbT["ID"][Y_out] = PwThat["ID"]["Y"]

We now add the calculated values to the (still) empty database `db` instantiated earlier.

#### Now we merge the first databases into the nesting tree objects. 

In [93]:
for module in modules:
    qS[module].index.name = "n"
    qD[module].index.name = "n"
    PwThat[module].index.name = "n"
    PwT[module].index.name = "n"
    PbT[module].index.name = "n"
    dbs[module]["qS"] = qS[module]
    dbs[module]["qD"] = qD[module]
    dbs[module]["PwThat"] = PwThat[module]
    dbs[module]["PbT"] = PbT[module]
    dbs[module]["mu"] = mu[module]

Merge `db` with the database attached to the nesting tree object `nt`:

In [94]:
for module in modules:
    DataBase.GPM_database.merge_dbs(nts[module].database, dbs[module], "first")

As a confirmation that this went succesfully, we check that the databse in `nt` contains a symbol called 'mu':

In [95]:
nts["ID"].database.series["mu"].vals.head()

n                 nn  
ID_1_electricity  ID_1    0.475
ID_1_oil          ID_1    0.475
ID_1_K            ID_1    4.050
C_EL_1            EL      0.050
U_ID_1_1          ID_1    0.500
Name: mu, dtype: float64

Success! We now proceed to use the class *abate*, a childclass of *gmspython*.

## Add sets used for calibration to the database

#### g_exo_vals consists of $\bar \sigma$ and $\bar \mu$. These are targets in the minimzation object

$\bar \sigma$ parameters used for the minimization object. These are the sigmas in the MNL nests (C->U)

In [96]:
sigmabar = {"ID":pd.Series([], name="sigmabar", dtype="float64"), "EOP":pd.Series([], name="sigmabar", dtype="float64")}
for c in output["ID"]["components"].keys():
    sigmabar["ID"][c] = 1

$\bar \mu$ parameters used for minimization object. These are the share parameters in the CET split from technologies to technology goods.

In [97]:
mubar = {"ID":mu["ID"].loc[pd.IndexSlice[mu["ID"].index.get_level_values(0).isin(flatten_list(list(output["ID"]["techs"].values()))), \
                     mu["ID"].index.get_level_values(1).isin(list(output["ID"]["techs"].keys()))]]}
mubar["ID"].name = "mubar"

#### g_tech_endo consists of parameters that are endogenized in the calibration procedure

In [98]:
mu_EC = mu["ID"].loc[pd.IndexSlice[pd.Series(mu["ID"].index.get_level_values(0).str.startswith("C")), \
                     pd.Series(mu["ID"].index.get_level_values(1).str.startswith("E"))]].index

mu_IOtech = mu["ID"].loc[pd.IndexSlice[mu["ID"].index.get_level_values(0).str.startswith("IO"), mu["ID"].index.get_level_values(0).str.startswith("IO")]].index

sigma_CU = sigmabar["ID"].index
sigma_CU.name = "n"

mu_CU = mu["ID"].loc[mu["ID"].index.get_level_values(0).isin(flatten_list(output["ID"]["components"].values())), mu["ID"].index.get_level_values(1).isin(output["ID"]["components"].keys())].index
mu_CU.name = "n"

mu_tautoU = mu["ID"].loc[pd.IndexSlice[mu["ID"].index.get_level_values(0).str.startswith("U"), mu["ID"].index.get_level_values(1).isin(output["ID"]["techs"].keys())]].index

Share-parameters of Leontief technologies (to be kept exogenous throughout)

In [99]:
t = pd.IndexSlice[pd.Series(mu["ID"].index.get_level_values(1).isin(output["ID"]["techs"].keys())), pd.Series(mu["ID"].index.get_level_values(1).isin(output["ID"]["techs"].keys()))]

t2 = pd.IndexSlice[pd.Series(mu["ID"].index.get_level_values(1).isin(output["ID"]["baseline_U_inputs"].keys())), \
                   pd.Series(mu["ID"].index.get_level_values(1).isin(output["ID"]["baseline_U_inputs"].keys()))]

mu_leontief_techs = mu["ID"].loc[t2].index.append(mu["ID"].loc[t].index)

# ENDO IN CALIB UDVIDES

In [100]:
tech_endoincalib_mu = {"ID":mu_EC.append(mu_IOtech).append(mu_CU).append(mu_tautoU), "EOP":None}
tech_endoincalib_sigma = {"ID":sigma_CU, "EOP":None}

g_endovars_exoincalib includes the endogenous variables that we exogenize in calibration because we have data on them (C due to potentials, sum of U due to current applications, sum of dX for IO tech and replacing tech due to IO data) 

The sumU and sumX objects are constructed because we need to calibrate to those. sumU hits current application and sumX hits IO input use data.

In [101]:
sumU = {"ID":multiindex_series(idx_level_names=["n", "nn"], series_name="sumU"), "EOP":multiindex_series(idx_level_names=["n", "nn"], series_name="sumU")}
for module in modules:
    for t in output[module]["techs"]:
        for U in output[module]["techs"][t]:
            C = find_key_from_value(output[module]["components"], U)
            upper = find_key_from_value(output[module]["upper_categories"], C)
            sumU[module][("sumU_" + t + "_" + upper, U)] = qD[module][U]

In [102]:
sumU_map = {"ID":sumU["ID"].index, "EOP":sumU["EOP"].index}

In [103]:
sumU_calibvalues = sumU.copy()
for module in modules:
    sumU_calibvalues[module].index = sumU_calibvalues[module].index.droplevel(1)
    sumU_calibvalues[module] = sumU_calibvalues[module].groupby("n").sum()

In [104]:
sumUaggs = {}
sumXaggs = {}
for module in modules:    
    sumUaggs[module] = sumU_map[module].get_level_values(0).unique()
    sumXaggs[module] = sumX2X[module].get_level_values(0).unique()

In [105]:
sumXaggs["EOP"]

Index(['sum_K', 'sum_electricity', 'sum_inp3', 'sum_inp4', 'sum_inp5'], dtype='object', name='n')

In [106]:
# endovars_exoincalib_sumU = sumU_map.copy()
# endovars_exoincalib_sumX = sumX
endovars_exoincalib_C = {"ID":mu_EC.get_level_values(0), "EOP":None}

In [107]:
#calib_values_currentapplications = pd.concat([qD[qD.index.str.startswith("U") & ~qD.index.str.contains("EOP")], qD[qD.index.str.startswith("C")]]) 

In [108]:
# calib_values_potentials = mu.loc[pd.IndexSlice[pd.Series(mu.index.get_level_values(0).str.startswith("C")), \
#                                  pd.Series(mu.index.get_level_values(1).str.startswith("E"))]]
calib_values_potentials = {}
for module in modules:
    calib_values_potentials[module] = output[module]["coverage_potentials"]

Initialize dict: tech_dbs with databases for ID and EOP

In [109]:
tech_dbs = {"ID":DataBase.GPM_database(), "EOP":DataBase.GPM_database()}
alwaysexo_mu = {"ID":mu["ID"].drop(tech_endoincalib_mu["ID"]).index, "EOP":mu["EOP"].index}

In [110]:
for module in modules:
    tech_dbs[module]["PwT"] = PwT[module] #New basic prices
    #db["endovars_exoincalib_sumU"] = endovars_exoincalib_sumU
    #db["endovars_exoincalib_sumX"] = endovars_exoincalib_sumX
    tech_dbs[module][module + "_" +"params_alwaysexo_mu"] = alwaysexo_mu[module]
    tech_dbs[module]["calib_values_currentapplications"] = sumU_calibvalues[module]
    tech_dbs[module]["calib_values_potentials"] = calib_values_potentials[module]
    if module == "ID":
        tech_dbs[module][module + "_" + "endovars_exoincalib_C"] = endovars_exoincalib_C[module]
        tech_dbs[module][module + "_" +"tech_endoincalib_mu"] = tech_endoincalib_mu[module]
        tech_dbs[module][module + "_" +"tech_endoincalib_sigma"] = tech_endoincalib_sigma[module]

In [111]:
#Minimization stuff
for module in modules:
    tech_dbs[module]["minobj"] = 1
    if module == "ID":
        tech_dbs[module]["minobj_mu"] = mubar[module]
        tech_dbs[module]["minobj_sigma"] = sigmabar[module]
        tech_dbs[module]["minobj_mu_subset"] = mubar[module].index
        tech_dbs[module]["minobj_sigma_subset"] = sigmabar[module].index
        tech_dbs[module]["weight_sigma"] = 1
        tech_dbs[module]["weight_mu"] = 1
    if module == "EOP":
        tech_dbs[module]["weight_sigmaG"] = 1
        tech_dbs[module]["weight_muG"] = 1


In [112]:
for module in modules:
    tech_dbs[module][module + "_" +"sumUaggs"] = sumU_map[module].get_level_values(0).unique()
    tech_dbs[module][module + "_" +"sumU2U"] = sumU_map[module]
    tech_dbs[module]["qsumU"] = sumU_calibvalues[module]
#     dbs[module]["qsumU"].name = "qsumU"

In [113]:
#sumXaggs set is shared between ID and EOP
tech_dbs["ID"]["sumXaggs"] = sumXaggs["ID"]

# for module in modules:
#     tech_dbs[module][module + "_" + "sumXaggs"] = sumXaggs[module]
   

In [114]:
for module in modules:
    tech_dbs[module]["sumX2X"] = sumX2X[module] #We give them the same name in the database here so they will end up being merged
#ID gets it own as well so we can move freely between ID and ID/EOP:
tech_dbs["ID"]["ID_sumX2X"] = sumX2X["ID"]

In [115]:
qsumX = {"ID":pd.Series([], name="qsumX", dtype="float64"), "EOP":pd.Series([], name="qsumX", dtype="float64")}
for module in modules:
    for sumx in sumX2X[module].get_level_values(0).unique():
        total = 0
        for x in sumX2X[module][sumX2X[module].get_level_values(0).isin([sumx])].get_level_values(1):
            total += qD[module][x]
        qsumX[module][sumx] = total
    qsumX[module].index.name = "n"

In [116]:
# sumX2X[module].loc[pd.IndexSlice[pd.Series(sumX2X[module].get_level_values(0).isin([sumx]))]]
# pd.IndexSlice[pd.Series(mu["ID"].index.get_level_values(1).isin(sumx)), pd.Series(mu["ID"].index.get_level_values(1).isin(output["ID"]["techs"].keys()))]

In [117]:
for module in modules:
    tech_dbs[module]["qsumX"] = qsumX[module]

Emission accounts stuff:

In [118]:
#Add the sumX-variable to the emission intensity variable (phi)

for m in output["EOP"]["upper_categories"].keys():
    for sumx in sumXaggs["ID"]:
        if sumx[4:] in emission_intensities.keys():
            if m in emission_intensities[sumx[4:]]:
                phi[(m, sumx)] = emission_intensities[sumx[4:]][m]
                continue
        phi[(m, sumx)] = default_emsint
#Update index names of M0 and M
M0.index.name = "n"
M.index.name = "n"

Adjust the input price for emission taxes:

In [119]:
#Store in database
tech_dbs["ID"]["phi"] = phi
tech_dbs["ID"]["map_M2X"] = phi.index
tech_dbs["ID"]["M0"] = M0
tech_dbs["EOP"]["M"] = M
tech_dbs["ID"]["M_subset"] = M0.index

In [120]:
#mapping that shows which components in EOP contribute to the abatement of which emission types
map_M2C = multiindex_series(idx_level_names=["n", "nn"], idx_name="map_M2C")
for m in output["EOP"]["upper_categories"]:
    for c in output["EOP"]["upper_categories"][m]:
        map_M2C[(m, c)] = np.nan
map_M2C = map_M2C.index

In [121]:
tech_dbs["EOP"]["map_M2C"] = map_M2C

In [122]:
#technical parameters in G-functions (muG and sigmaG)
muG = pd.Series([], name="muG", dtype="float64")
sigmaG = pd.Series([], name="sigmaG", dtype="float64")
for c in map_M2C.get_level_values(1):
    muG[c] = 0
    sigmaG[c] = 1


muG.index.name = "n"
sigmaG.index.name = "n"
# EOP_C_subset = muG.index

In [123]:
# EOP_C_subset.name = "n"
# tech_dbs["EOP"]["EOP_C_subset"] = EOP_C_subset

In [124]:
#Potentials
theta = calib_values_potentials["EOP"].copy()
theta.index = theta.index.droplevel(1)
theta.name = "theta"

In [125]:
#Calibration procedure unfixes all muG and sigmaG.
muGbar = muG.copy()
sigmaGbar = sigmaG.copy()

In [126]:
#add to database
module = "EOP"
tech_dbs[module]["pM"] = pM
tech_dbs[module]["muG"] = muG
tech_dbs[module]["sigmaG"] = sigmaG
tech_dbs[module]["minobj_muG"] = muGbar
tech_dbs[module]["minobj_sigmaG"] = sigmaGbar
tech_dbs[module]["theta"] = theta


module = "ID"
tech_dbs[module]["pMhat"] = pMhat


Finally, we adjust the prices according to the emission content of them

# **3: The model**

We now build a gams model using a *gmspython* childclass (*abate*). We build the partial equilibrium model from the following principles:
* The firm takes input prices as given. These are prices with taxes ('PwThat') that are defined over the global set *inp*. In this case this includes e.g. the price on electricity and capital.
* The firm further takes the quantity of supply as given (it has to be either this or output prices for the model to be square). We denote the quantity of supply 'qS', and define it over the subset of goods that are outputs from the sector *out*. 
* The firm's demand for intermediate goods, and their prices are endogenous to the firm: In our case this involves $\tau$, $U$ and $C$ in ID and only $\tau$ and $U$ in EOP. These intermediate goods are not traded on any market, but is simply a construct we use to build the nests. In this way the quantity of $U$ is both supplied, and demanded by the firm/sector itself. As we do not need both a supply and demand variable (they are the same in this case), we let the quantity/prices for intermediate goods go under the variables 'qD'/'PwThat', defined over the subset *int*.
* The demand for inputs is also endogenous to the firm. We denote this 'qD' defined over the subset of goods 'inp'.
* As briefly mentioned above, we either have to let the output prices / quantities be endogeonous (and the other exogenous) for the module to be square. In this case we let the price on outputs be endogenous. We denote the price 'PbT' for price before taxes. This is defined over the subset *out*. Distinguishing between PbT and PwThat allows for the flexibility of including a tax module / including a mark-up on profits at a later point.

We will get back to the way we initialize the *gmspython* module next; here we initialize the class to be able to print the variables needed for this specific model.
The model, denoted `m`, is an instance of the class `abate` which is itself a childclass of the more general `gmspython` class.

In [127]:
m = abatement.abate(nt=nts["ID"], tech_db=tech_dbs["ID"], work_folder=work_folder, use_EOP={"tech_db":tech_dbs["EOP"], "nt":nts["EOP"]}, **{'data_folder': gams_folder, "name":"Abatement"})

In [128]:
m.model.database.update_all_sets(clean_up=False) #Makes n include the sum variables

In [129]:
m.model.functions = {"std_pdf":"$FUNCTION std_pdf({x}): ((1/(sqrt(2*Pi)))*exp(-(Sqr({x}))/2)) $ENDFUNCTION"}

Recall that running a model requires constructing three methods/properties for `m`:

1. `m.initialize_variables()`
2. `m.endo_groups` and `m.exo_groups`
3. `m.add_blocks()`

We briefly review each in turn.

`m.initialize_variables()` sets default initial values for the variables and parameters that are not already present in the attached database (which we constructed earlier). For example, we did not specify the values for the mark-up parameters anywhere, so the method `initialize_variables()` *must* specify the value that these parameters should have. We can see that in this case they will simply be set to 1 (no markup):

The default values for markups are zero, i.e. no markups:

We run the method using the keyword argument `check_variables:True`. This makes the program check whether the database contains all the necessary values of a specific variable/parameter (say, $\mu$), and if not, merges the default value (in this case equal to 1) onto the series of share-parameters already contained in the database. 

In [130]:
# m.use_EOP = False

Since all of our starting values were set 'as if' the entire tree was Leontief, it is worth noting that the default values for substitutions of elasticity/transformation are 1, e.g. for $\sigma$:

In [131]:
m.default_var_series("sigma").head()

n
IO_tech             0.0001
U_ID_C_EL_2_base    0.0001
Y                   0.0001
ER                  0.0001
C_EL_4              0.0001
Name: sigma, dtype: float64

The second requirement(s) are the properties `endo_groups` and `exo_groups`. These collect the variables in groups and specify which of these subgroups are part of the two upper groups of endogenous and exogenous variables/parameters respectively. The choice of whether a variable should be endogenous or exogenous changes e.g. depending on whether the model is about to be solved for calibration purposes or not.
In the basic case (not calibration mode), the endogenous variables are split into multiple groups:

In [132]:
m.endo_groups.keys()

dict_keys(['Abatement_g_ID_prices_alwaysendo', 'Abatement_g_ID_quants_alwaysendo', 'Abatement_g_emissions_alwaysendo', 'Abatement_g_EOP_prices_alwaysendo', 'Abatement_g_prices_endogenouswithEOP', 'Abatement_g_EOP_quants_alwaysendo', 'Abatement_g_emissions_endoinEOP', 'Abatement_g_ID_quants_exoincalib', 'Abatement_g_EOP_quants_exoincalib'])

... which themselves include the actual variables. Here we print the values of prices with taxes in the endovars group (the first 5 only). This uses the method *.var_endo*.

In [133]:
m.initialize_variables()

In [134]:
m.add_groups()

In [135]:
m.add_blocks()

In [136]:
m.var_exo("qS")

n
Y_out_electricity    160.0
Y_out_inp3           160.0
Y_out_inp4           160.0
Y_out_inp5           160.0
Y_out_oil            160.0
Name: qS, dtype: float64

And next we print the exogenous prices with taxes by doing the same thing, but for exogenous groups instead. This is found in the var_exo method:

Finally, we need to specify the *add_blocks* method. This is the method that writes blocks of equations to the model. In our case, we write a different block of equations for each tree.
For example, a set of equations are:

In [137]:
print(m.eqtext(list(m.ns_local)[0]))

E_zp_out_ID_EC[n]$(ID_out_ID_EC[n])..	PbT[n]*qS[n] =E= sum(nn$(map_ID_EC[nn,n]), qD[nn]*PwThat[nn]);
	E_zp_nout_ID_EC[n]$(kno_no_ID_EC[n])..	PwThat[n]*qD[n] =E= sum(nn$(map_ID_EC[nn,n]), qD[nn]*PwThat[nn]);
	E_q_out_ID_EC[n]$(bra_o_ID_EC[n])..	qD[n] =E= sum(nn$(map_ID_EC[n,nn]), mu[n,nn] * (PbT[nn]/PwThat[n])**(sigma[nn]) * qS[nn] / sum(nnn$(map_ID_EC[nnn,nn]), mu[nnn,nn] * (PbT[nn]/PwThat[nnn])**(sigma[nn])));
	E_q_nout_ID_EC[n]$(bra_no_ID_EC[n])..	qD[n] =E= sum(nn$(map_ID_EC[n,nn]), mu[n,nn] * (PwThat[nn]/PwThat[n])**(sigma[nn]) * qD[nn] / sum(nnn$(map_ID_EC[nnn,nn]), mu[nnn,nn] * (PwThat[nn]/PwThat[nnn])**(sigma[nn])));


#### Model summary:

To sum up, the model we have in mind here has the following main settings (where a \$ denotes a condition):
* Endogenous variables: $PwThat\$(int)$, $PbT\$(out)$, $qD\$(wT)$. (Recall that the subset wT is the union of intermediate goods and inputs)
* Exogenous variables: $PwThat\$(inp)$, $qS\$(out)$. 
* Equations: For the CES nest we have the following two equations:
    $$\begin{align}
        q_j =& \mu_j\left(\dfrac{p_j}{p_i}\right)^{-\sigma}q_i, \tag{CES-1}\\
        p_iq_i =& \sum_j q_jp_j \tag{CES-2}
    \end{align}$$
    (CES-1) is the CES demand function that has to hold for all *branches* ($q_j$) where $q_i$ is the relevant knot in the nesting tree. (CES-2) is a zero-profit condition that has to hold for production functions with constant returns to scale (alternatively, we can use a price index). This has to hold for every *knot* where $j$ sums over the relevant branches. A similar thing has to hold for the CET tree.

### **Running the model**
Running the model, including the invoking the methods and properties described in the previous section, we simply run the following:

In [138]:
m.write_and_run()

To check whether the model was successfully solved, we check the modelstat (16.0 means solved correctly, 5.0 means not)

In [139]:
m.model_instances["baseline"].__dict__

{'execute_name': 'CollectAndRun.gms',
 'name': 'gmodel',
 'settings': <DB2Gams_l2.gams_settings at 0x228f7a83b48>,
 'export_settings': {'dropattrs': ['settings', 'opt', 'job'],
  'pklattrs': {'settings': 'gams_settings'},
  'opt': 'conopt4.opt'},
 'opt': <gams.options.GamsOptions at 0x228f7a834c8>,
 'opt_file': 'conopt4.opt',
 'import_settings': {},
 'job': <gams.execution.GamsJob at 0x228f7ab2688>,
 'out_db': <DataBase.GPM_database at 0x228f7a83948>,
 'modelstat': 16.0,
 'solvestat': 1.0}

Run in the ID state:

In [140]:
m.setstate("ID")

In [141]:
m.reset_settings()

In [142]:
m.state

'ID'

In [124]:
m.initialize_variables(**{"check_variables":True})

In [125]:
m.add_blocks()

In [126]:
m.add_groups()

In [127]:
m.reset_settings()

In [128]:
m.write_and_run()

In [112]:
m.model_instances["baseline"].modelstat

16.0

In [113]:
m.setstate("EOP")

In [114]:
m.reset_settings()

In [115]:
m.write_and_run()

In [116]:
m.model_instances["baseline"].modelstat

16.0

The model does not run when all sigmas are not set to be Leontief, i.e. when they use their default value of 1.

We wish to solve the model while instead of the default value of substitution and transformation elasticities of 0.1, we set them equal to zero (in practice not exactly, but very close).
Since the starting values have been set under the assumption of Leontief nests, this will help the solver find an initial solution.
Since EOP is not consistent with Leontief, these elasticities specifically must not be zero. To make sure we will always find a solution, we choose a high number, e.g. 2:

In [117]:
# m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = 0.0001
# m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -0.0001
#m.get("sigma")[:] = 0.0001
# m.get("sigma")[m.get("EOP_C_subset")] = 2
#m.get("eta")[:] = -0.0001
# m.get("eta")[m.get("eta").index.str.startswith("EOP")] = -2

#m.model.settings.databases["Abatement"]["sigma"].vals[condition] = 0.2
#m.write_and_run(options_run={'output':sys.stdout})
m.write_and_run()
if m.model_instances["baseline"].modelstat == 16.0:
    print("\nSuccess! The modelstat was 16.0")


Success! The modelstat was 16.0


Save model by pickling it, using the export method:

In [118]:
m.export()

'C:\\Users\\zgr679\\Documents\\GitHub\\GPM_v05\\examples\\Abatement\\Data\\..\\gamsmodels\\Main\\gmspython_Abatement'

The model is calibrated in the next jupyter file.